In [12]:
import json, re, sys, warnings
from pathlib import Path
from collections import Counter

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect, LangDetectException

warnings.filterwarnings("ignore")               # menos ruido
sns.set_theme(style="whitegrid", font_scale=.9)



In [15]:
FILE = Path("out_concat.jsonl")
FIG_DIR = Path("figs")
CSV_DIR = Path("csv")
MAX_WORDS   = 512
HALLUC_PAT  = re.compile(r"(model que:){3,}", re.IGNORECASE)  # ≥3 repeticiones
HALLU_RE   = re.compile(r"(model que:){3,}", re.IGNORECASE)   # ≥ 3 repeticiones

FIG_DIR.mkdir(exist_ok=True)
CSV_DIR.mkdir(exist_ok=True)



In [16]:
# -------------------- 1. Cargar y validar JSONL ---------------------
rows, bad_json = [], 0
with FILE.open(encoding="utf-8") as f:
    for ln in f:
        ln = ln.strip()
        if not ln:
            continue
        try:
            rows.append(json.loads(ln))
        except json.JSONDecodeError:
            bad_json += 1

print(f"✔️  Registros válidos: {len(rows):,} · JSON inválido: {bad_json:,}")
df = pd.DataFrame(rows)

# -------------------- 2. Conteo de palabras -------------------------
for col in ("generated_solution", "generated_secondary_answer"):
    if col not in df.columns:
        df[col] = None
    df[f"{col}_word_count"] = df[col].fillna("").str.split().str.len()

long_mask = (
    (df["generated_solution_word_count"]        > MAX_WORDS) |
    (df["generated_secondary_answer_word_count"] > MAX_WORDS)
)
df_long  = df[long_mask]

# ---------------- 3. Detección de “alucinados” ----------------------
def hallu(txt: str | None) -> bool:
    return bool(txt and HALLU_RE.search(txt))

df["hallucinated"] = df["generated_solution"].apply(hallu)
df_hallu = df[df["hallucinated"]]


✔️  Registros válidos: 16,862 · JSON inválido: 0


In [17]:

# ---------------- 4. Verificación de idioma (español) ---------------
def detect_lang(txt: str | None) -> str:
    try:
        return detect(txt) if txt and txt.strip() else "und"
    except LangDetectException:
        return "err"

df["lang"]       = df["generated_solution"].apply(detect_lang)
df["is_spanish"] = df["lang"].eq("es")
df_not_es        = df[~df["is_spanish"]]

# ---------------- 5. Normalizar «subject» ---------------------------
CATEGORIAS_CANONICAS = {
    "Operaciones básicas": [
        "suma","resta","adición","sustracción","multiplicación","división"
    ],
    "Números y porcentajes": [
        "números","enteros","fracciones","decimales","porcentajes","fractions"
    ],
    "Geometría básica": ["geométricas","figuras","perímetro","área"],
    "Medición": ["medición","volumen","longitud","tiempo","unidad"],
    "Datos y gráficos": ["datos","tablas","gráficos"],
    "Igualdades y desigualdades": ["igualdades","desigualdades","comparación"],
    "Reconocimiento de patrones": ["patrones","regularidades"],
    "Problemas con contexto real": ["vida real","contexto","cotidiano","palabras"],
}

def map_subject(text: str | None) -> str:
    if not isinstance(text, str):
        return "INVALIDO"
    t = text.lower()
    for cat, kws in CATEGORIAS_CANONICAS.items():
        if any(re.search(rf"\b{re.escape(kw)}\b", t) for kw in kws):
            return cat
    return "INVALIDO"

df["subject_normalizado"] = df["subject"].apply(map_subject)
df_invalid_subj           = df[df["subject_normalizado"] == "INVALIDO"]

# ---------------- 6. Resumen de descartes ---------------------------
summary = {
    "total_leidos":               len(rows) + bad_json,
    "json_invalidos":             bad_json,
    "total_registros":            len(df),
    "null_generated_solution":    df["generated_solution"].isna().sum(),
    "null_generated_secondary":   df["generated_secondary_answer"].isna().sum(),
    "exceso_longitud":            int(long_mask.sum()),
    "patrones_alucinados":        len(df_hallu),
    "no_espanol":                 len(df_not_es),
    "subject_INVALIDO":           len(df_invalid_subj),
}

pd.DataFrame(summary.items(), columns=["métrica", "conteo"])\
  .to_csv(CSV_DIR / "resumen_calidad.csv", index=False)

# ---------------- 7. Dataset limpio final ---------------------------
clean_mask = (
    (~long_mask) &
    (~df["hallucinated"]) &
    (df["is_spanish"]) &
    (df["subject_normalizado"] != "INVALIDO")
)
df_clean = df[clean_mask].copy()
print(f"✅ Registros finales para fine-tuning: {len(df_clean):,}")

# =================== 8. GRÁFICAS Y REPORTES =========================
## 8-A  Descartes por motivo
plt.figure(figsize=(8,4))
plt.bar(summary.keys(), summary.values(), color="indianred")
plt.title("Registros descartados por motivo")
plt.ylabel("Cantidad")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(FIG_DIR / "descartes_por_motivo.png", dpi=150)
plt.close()

## 8-B  Distribución de subject_normalizado
plt.figure(figsize=(8,4))
sns.countplot(data=df_clean,
              y="subject_normalizado",
              order=df_clean["subject_normalizado"].value_counts().index,
              palette="deep")
plt.title("Distribución de «subject_normalizado» (dataset limpio)")
plt.xlabel("Frecuencia"); plt.ylabel("Categoría")
plt.tight_layout()
plt.savefig(FIG_DIR / "subjects_dataset_limpio.png", dpi=150)
plt.close()

## 8-C  Distribución de problem_source  # <<< NEW >>>
if "problem_source" in df_clean.columns:
    counts_ps = df_clean["problem_source"].value_counts()
    # CSV con la distribución
    counts_ps.to_csv(CSV_DIR / "dist_problem_source.csv", header=["frecuencia"])
    # gráfica
    plt.figure(figsize=(8,4))
    sns.barplot(x=counts_ps.values, y=counts_ps.index, palette="muted")
    plt.title("Distribución de «problem_source» (dataset limpio)")
    plt.xlabel("Frecuencia"); plt.ylabel("problem_source")
    plt.tight_layout()
    plt.savefig(FIG_DIR / "problem_source_dist.png", dpi=150)
    plt.close()
else:
    print("⚠️  Columna «problem_source» no encontrada; se omite la gráfica.")

## 8-D  Histogramas de longitud
for col, ttl in [("generated_solution_word_count",  "Longitud solución"),
                 ("generated_secondary_answer_word_count","Longitud respuesta secundaria")]:
    plt.figure(figsize=(8,4))
    sns.histplot(df_clean[col], bins=20, kde=True, color="steelblue")
    plt.title(f"{ttl} (palabras)")
    plt.xlabel("Número de palabras")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{col}_hist.png", dpi=150)
    plt.close()

# ---------------- 9. EXPORTAR subconjuntos --------------------------
df_clean.to_json(CSV_DIR / "out_concat_clean.jsonl",
                 orient="records", lines=True, force_ascii=False)
df_long.to_json(CSV_DIR / "descartes_exceso_longitud.jsonl",
                orient="records", lines=True, force_ascii=False)
df_hallu.to_json(CSV_DIR / "descartes_alucinados.jsonl",
                 orient="records", lines=True, force_ascii=False)
df_not_es.to_json(CSV_DIR / "descartes_no_espanol.jsonl",
                  orient="records", lines=True, force_ascii=False)
df_invalid_subj.to_json(CSV_DIR / "descartes_subject_inval.jsonl",
                        orient="records", lines=True, force_ascii=False)



✅ Registros finales para fine-tuning: 14,315
